In [ ]:
# creating lists of paths to photos and encodings
import pathlib
input_path = pathlib.Path('data/train/input')
input_list = sorted([str(path) for path in input_path.glob('*.jpg')])

enc_path = pathlib.Path('data/train/enc')
enc_list = sorted([str(path) for path in enc_path.glob('*.jpg')])
for i in range(12):
    print(input_list[i], enc_list[i])

In [ ]:
# creating paris of phots and encodings (0 - photos in pairs is not the same, 1- photo and encoding are the same picture)
import random
def create_list_of_pairs_and_labels(input_list, enc_list, number_of_not_correct_pairs):
    step = number_of_not_correct_pairs+1
    n_data = step*len(input_list)
    pairs = [0]*n_data
    labels = [0]*n_data
    for photo_idx in range(0, len(input_list)):
        pairs[photo_idx*step] = [input_list[photo_idx], enc_list[photo_idx]]
        labels[photo_idx*step] = 1 # pair of photo and its encoding
        enc_without_correct_pair = [x for i, x in enumerate(enc_list) if i != photo_idx]
        for pair_idx in range(1,number_of_not_correct_pairs+1):
            enc = random.choice(enc_without_correct_pair)
            enc_without_correct_pair.remove(enc)
            pairs[photo_idx*step+pair_idx] = [input_list[photo_idx], enc]
            labels[photo_idx*step+pair_idx] = 0 # pair of photo and encoding of some other photo
    return pairs, labels
data = create_list_of_pairs_and_labels(input_list, enc_list, 1)

In [ ]:
# classes for PyTorch trainig
from torch.utils.data import Dataset
from PIL import Image
from torchvision.transforms import transforms

class ImageDataset(Dataset):
    def __init__(self, pairs, labels, transform=None):
        self.pairs = pairs
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        photo = Image.open(self.pairs[idx][0])
        encoding = Image.open(self.pairs[idx][1])
        if self.transform is not None:
            photo = self.transform(photo)
            encoding = self.transform(encoding)
        return photo, encoding, self.labels[idx]

pairs, labels = data
transform = transforms.Compose([
    transforms.ToTensor()
])
ds = ImageDataset(pairs, labels, transform)


In [ ]:
# todo: zrobić clasę modelu (książka - skończyłem na 398 stronie